In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 29.4 ms, sys: 12.2 ms, total: 41.6 ms
Wall time: 618 ms


In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

In [4]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-20/Mask_ERF_iris-20.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [5]:
track = pd.read_csv('../saved-files/threshold-20/track-reset-20.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,115.000000,211.000000,1,20,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.175000e+06,128000.000000,70.390411,13.368005,1,0 days 00:00:00
1,0,2,133.423757,399.000000,2,20,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193424e+06,316000.000000,70.050984,18.198025,2,0 days 00:00:00
2,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00
3,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00
4,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15538,299,21,259.000000,321.000000,1,20,15517,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.319000e+06,238000.000000,69.002419,15.859776,3843,0 days 00:00:00
15539,300,18,259.974309,323.199741,5,20,15539,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.319974e+06,240199.740987,68.991524,15.911087,3843,0 days 00:05:00
15540,299,22,274.046384,288.604740,4,20,15518,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.334046e+06,205604.739833,68.893877,15.034156,3844,0 days 00:00:00
15541,300,20,274.932709,290.064315,17,20,15541,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.334933e+06,207064.314866,68.884654,15.067793,3844,0 days 00:05:00


In [6]:
# # Changing time cell to timedelta
# time_cell = track['time_cell'].astype('timedelta64[ns]').values
# time_cell

### CALCULATING FOR DISTANCE

In [7]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of the radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of where the radar is located
    
# 69.2412° N, 16.0030° E
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [8]:
distance[:10]
# np.max(distance)

[163.09069056111275,
 123.87500863978714,
 147.5867183439574,
 144.15393694654776,
 141.54493121094782,
 138.46093238167992,
 102.95380270930283,
 103.34082399479416,
 53.03655548072585,
 64.09751022660994]

In [9]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 2.7 ms, sys: 0 ns, total: 2.7 ms
Wall time: 2.7 ms


In [10]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,1,115.000000,211.000000,1,20,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.175000e+06,128000.000000,70.390411,13.368005,1,0 days 00:00:00,163.090691
1,0,2,133.423757,399.000000,2,20,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193424e+06,316000.000000,70.050984,18.198025,2,0 days 00:00:00,123.875009
2,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00,147.586718
3,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00,144.153937
4,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00,141.544931


## _cells that lived for at least 10 mins old_ 

In [11]:
%%time
# cell_one_hour = np.timedelta64( 600, 's') #10 minutes in seconds
cell_times = np.timedelta64( 600, 's') #10 minutes in seconds

#convert time_cell to seconds
time_cell = track['time_cell'].astype('timedelta64[ns]').values

# track cell that live for at least 10 minutes long
track_close_to_AMF1 = track[ time_cell >= cell_times ]
# track_close_to_AMF1 = track[ ((track['distance'] <= 95) & (track['time_cell'].values >= cell_one_hour)) ]
track_close_to_AMF1= track_close_to_AMF1.drop_duplicates(subset=['cell']) # remove duplicates of cell_ids
track_close_to_AMF1.head()

CPU times: user 35.7 ms, sys: 1.88 ms, total: 37.6 ms
Wall time: 65.5 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
4,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00,141.544931
11,2,5,186.543783,362.000000,2,20,109,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.246544e+06,279000.000000,69.616406,17.079360,7,0 days 00:10:00,59.301956
14,2,6,195.334382,362.598927,12,20,110,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.255334e+06,279598.927410,69.536276,17.067031,8,0 days 00:10:00,53.078408
21,2,8,220.998380,347.062464,9,20,112,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.280998e+06,264062.464198,69.320971,16.603518,10,0 days 00:10:00,25.261703
26,2,9,241.956826,350.264995,8,20,113,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.301957e+06,267264.994733,69.127866,16.622583,12,0 days 00:10:00,27.565448


In [12]:
# total track cells found.
len(track_close_to_AMF1) 

1773

# tracking all cells that were at least 10 minutes old back in time to get all the cells

In [13]:
%%time
combined_tracks_AMF1 = []
for index, cell_id in enumerate(np.unique(track_close_to_AMF1['cell'].values)): # looped through tracks found with AMF1 site
    if cell_id in track['cell']: # checking if those cells(tracks) are within the original tracked dataframe
        track_i = track[track['cell'] == cell_id] # display it if it is within the tracked dataframe
        combined_tracks_AMF1.append(track_i) # append the results you get a new dataFrame calld combined_tracks_AMF1

# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
# tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1, keys=['df1', 'df2', 'df3', 'df4', 'df5', 'df6'])
tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1)
tracks_closer_AMF1

CPU times: user 836 ms, sys: 11.3 ms, total: 847 ms
Wall time: 1.74 s


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
2,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00,147.586718
3,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00,144.153937
4,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00,141.544931
5,3,3,148.635454,213.044500,3,20,148,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.208635e+06,130044.500153,70.083824,13.369686,3,0 days 00:15:00,138.460932
9,0,7,179.772123,360.749762,7,20,7,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.239772e+06,277749.761570,69.679007,17.069049,7,0 days 00:00:00,64.097510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15526,299,1,132.210510,264.802250,9,20,15497,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.192211e+06,181802.250223,70.199616,14.740749,3835,0 days 00:05:00,117.272271
15527,300,1,134.016782,265.607027,8,20,15522,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.194017e+06,182607.027340,70.182629,14.757736,3835,0 days 00:10:00,115.288355
15529,298,15,209.325817,289.439165,8,20,15481,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.269326e+06,206439.164789,69.480737,15.197855,3837,0 days 00:00:00,41.339728
15530,299,12,210.187722,291.164259,7,20,15508,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.270188e+06,208164.259129,69.471488,15.239071,3837,0 days 00:05:00,39.445606


In [14]:
# resetting the index
tracks_closer_AMF1_index_reset = tracks_closer_AMF1.reset_index(drop=True)
tracks_closer_AMF1_index_reset.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00,147.586718
1,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00,144.153937
2,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00,141.544931
3,3,3,148.635454,213.044500,3,20,148,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.208635e+06,130044.500153,70.083824,13.369686,3,0 days 00:15:00,138.460932
4,0,7,179.772123,360.749762,7,20,7,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.239772e+06,277749.761570,69.679007,17.069049,7,0 days 00:00:00,64.097510


In [15]:
# len(tracks_closer_AMF1_index_reset)

# Algorithm to calculate the corresponding reflectivity values (calculate for the max and mean dBZ) and the area

In [16]:
unique_cell_ids = track_close_to_AMF1['cell'].values # getting the cells_tracked
len(unique_cell_ids)

1773

In [17]:
# len(np.unique(tracks_closer_AMF1_index_reset['cell']))
# Mask_ERF = iris.load('./saved-files/Mask_ERF_iris.nc')[0]
# Mask_ERF

In [18]:
%%time
# all masked cell in iris cube
masked_tracked_cell = []
for i, unique_value in enumerate(unique_cell_ids):
    # get masked cell based on cell id
    get_mask_cell = tobac.utils.mask_cell(mask=Mask_ERF, cell=unique_value, track=track, masked=False) 
    masked_tracked_cell.append(get_mask_cell)

CPU times: user 4.9 s, sys: 50.3 ms, total: 4.95 s
Wall time: 10.2 s


In [19]:
# masked_tracked_cell[:10]

In [21]:
%%time
# Convert the segmented/mask cells from iris cube to xrray
masked_values_to_xarray = []
for i, items in enumerate(masked_tracked_cell):
    mask_convert_xr = xr.DataArray.from_iris(masked_tracked_cell[i]) # convert iris cube to xarray data array
    masked_values_to_xarray.append(mask_convert_xr) # append the results to list
    # print(mask_convert_xr)

CPU times: user 3.73 s, sys: 24.3 ms, total: 3.76 s
Wall time: 7.79 s


In [22]:
%%time
# Getting the original masked reflectivity values
slice_cell_times = []
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id] 
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    # slice values based on time
    slice_cell_time = masked_values_to_xarray[i].sel(time=slice(first_time, last_time))
    slice_cell_times.append(slice_cell_time)
    # print(slice_cell_time)

CPU times: user 2.88 s, sys: 28.2 ms, total: 2.91 s
Wall time: 6.64 s


In [23]:
# slice_cell_times[:5]

In [24]:
# type(slice_cell_times)

In [25]:
# %%time
# # Create an empty Dataset
# dataset = xr.Dataset()

# # Add each DataArray to the Dataset as a separate variable
# for index, data_array in enumerate(slice_cell_times):
#     var_name = f'data_array_{index}'  # Create a variable name for each DataArray
#     dataset[var_name] = data_array  # Add the DataArray to the Dataset

# # Save the Dataset as a NetCDF file
# dataset.to_netcdf('all_data_arrays.nc')

In [26]:
%%time
equivalent_reflectivity_factors = 10**((equivalent_reflectivity_factor/10))
# equivalent_reflectivity_factors = 10*np.log10(Z)

CPU times: user 4.3 s, sys: 270 ms, total: 4.57 s
Wall time: 9.6 s


## algorithm to extract the speed, area, maximum, minimum and mean reflectivity values based on cell_id and specific times

In [27]:
%%time
min_reflectivities = []; mean_reflectivities = []; max_reflectivities = []; areas = [];
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    #################################
    import warnings
    warnings.filterwarnings('ignore')
    #################################
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id]  # dataframe based on cell_id
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    
#     # # # # Calculating area
    # area = tobac.analysis.calculate_area(track, Mask_ERF)
#     # area = tobac.analysis.calculate_area(cell_ids, masked_tracked_cell[i], method_area=None)
#     area = np.absolute(area['area']/1000000) # area in kilometers
#     areas.append(area)
    
    # area1 = np.divide(area['area'],1000000)
    # area['area($km^2$)'] = area1


    # reflectivity_masked based on times
    reflectivity_masked = equivalent_reflectivity_factors.sel(time=slice(first_time, last_time))
    
    # LOOPING THROUGH THE AND GETTING THE VALUES
    for index in (range(len(reflectivity_masked))):
        mean_ref = reflectivity_masked[index].mean().item() # get mean values of reflectivity # in dBZ values
        # # Convert from dBZ to Z values
        # mean_ref_Z = 10**(mean_ref / 10)
        # # Convert it back to dBZ units
        # mean_ref_dBZ = 10*np.log10(mean_ref_Z)
        #####################################
        max_ref = reflectivity_masked[index].max().item() # get max values of reflectivity
        
        mean_reflectivities.append(mean_ref)
        max_reflectivities.append(max_ref)
        
# concatenate the area values to one series
# area1 = pd.concat(areas)

# calculate velocity
# velocity = tobac.analysis.calculate_velocity(tracks_closer_AMF1_index_reset)

CPU times: user 23.1 s, sys: 53.3 ms, total: 23.2 s
Wall time: 47.7 s


In [28]:
# converting the mean Z unit back to dBZ
mean_reflectivities1 = 10*np.log10(mean_reflectivities)

# converting the max Z unit back to dBZ
max_reflectivities1 = 10*np.log10(max_reflectivities)

In [29]:
np.min(mean_reflectivities1), np.max(mean_reflectivities1)

(12.70891348332567, 17.0430868015557)

In [30]:
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00,147.586718
1,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00,144.153937
2,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00,141.544931
3,3,3,148.635454,213.044500,3,20,148,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.208635e+06,130044.500153,70.083824,13.369686,3,0 days 00:15:00,138.460932
4,0,7,179.772123,360.749762,7,20,7,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.239772e+06,277749.761570,69.679007,17.069049,7,0 days 00:00:00,64.097510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12803,299,1,132.210510,264.802250,9,20,15497,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.192211e+06,181802.250223,70.199616,14.740749,3835,0 days 00:05:00,117.272271
12804,300,1,134.016782,265.607027,8,20,15522,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.194017e+06,182607.027340,70.182629,14.757736,3835,0 days 00:10:00,115.288355
12805,298,15,209.325817,289.439165,8,20,15481,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.269326e+06,206439.164789,69.480737,15.197855,3837,0 days 00:00:00,41.339728
12806,299,12,210.187722,291.164259,7,20,15508,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.270188e+06,208164.259129,69.471488,15.239071,3837,0 days 00:05:00,39.445606


In [31]:
# append max min and mean reflectivity values based on each cell id and times to the tracks_closer_AMF1_index_reset dataframe
# tracks_closer_AMF1_index_reset['speed'] = velocity
# tracks_closer_AMF1_index_reset['area1'] = area1
tracks_closer_AMF1_index_reset['mean dBZ'] = pd.Series(mean_reflectivities1)
tracks_closer_AMF1_index_reset['max dBZ'] = pd.Series(max_reflectivities1)

In [32]:
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ
0,0,3,138.459946,209.639577,12,20,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198460e+06,126639.576617,70.178065,13.296810,3,0 days 00:00:00,147.586718,13.245867,34.567482
1,1,2,142.106890,211.038006,11,20,56,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.202107e+06,128038.005949,70.144203,13.327626,3,0 days 00:05:00,144.153937,12.974534,35.577388
2,2,3,144.716901,212.259109,4,20,107,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.204717e+06,129259.108875,70.119843,13.355321,3,0 days 00:10:00,141.544931,12.780750,34.063904
3,3,3,148.635454,213.044500,3,20,148,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.208635e+06,130044.500153,70.083824,13.369686,3,0 days 00:15:00,138.460932,12.708913,36.584485
4,0,7,179.772123,360.749762,7,20,7,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.239772e+06,277749.761570,69.679007,17.069049,7,0 days 00:00:00,64.097510,13.245867,34.567482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12803,299,1,132.210510,264.802250,9,20,15497,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.192211e+06,181802.250223,70.199616,14.740749,3835,0 days 00:05:00,117.272271,15.240493,41.116443
12804,300,1,134.016782,265.607027,8,20,15522,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.194017e+06,182607.027340,70.182629,14.757736,3835,0 days 00:10:00,115.288355,14.702386,40.106802
12805,298,15,209.325817,289.439165,8,20,15481,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.269326e+06,206439.164789,69.480737,15.197855,3837,0 days 00:00:00,41.339728,15.301029,39.603226
12806,299,12,210.187722,291.164259,7,20,15508,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.270188e+06,208164.259129,69.471488,15.239071,3837,0 days 00:05:00,39.445606,15.240493,41.116443


In [33]:
tracks_closer_AMF1_index_reset.to_csv('../saved-files/threshold-20/final-tracks-threshold-20-excluding-area.csv', index=False)

# END 

# Next Step : 

### Plot the area, max dBZ, mean dBz against time or distance.
## Do they intensify as it approaches the coast
### Perform statistics on cells
## Do the cells that merged/split?

### -----------------------------

## distance to landfall.  (2)
# start with time to landfall --- (1)